# Evaluate the performance of LAMAR-SS in prediction of splice site

In [1]:
from LAMAR.modeling_nucESM2 import EsmForTokenClassification
from transformers import AutoConfig, AutoTokenizer, DataCollatorForTokenClassification, TrainingArguments, Trainer
from safetensors.torch import load_file, load_model
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import os
import pandas as pd
import torch
from torch import nn
import numpy as np
import json
import tqdm

/opt/anaconda3/envs/torch201/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
os.chdir('/picb/rnasys2/zhouhanwen/github/LAMAR/')

In [3]:
tokenizer_path = 'tokenizer/single_nucleotide/'
model_max_length = 1026
model_name = 'config/config_150M.json'
token_dropout = False
positional_embedding_type = 'rotary'
hidden_size = 768
intermediate_size = 3072
num_attention_heads = 12
num_hidden_layers = 12
nlabels = 3
model_state_path = 'SpliceSitePred/saving_model/mammalian_4096/bs128_lr1e-4_wr0.05_4epochs/checkpoint-20000/model.safetensors'
test_set_path = 'SpliceSitePred/data/gencode_test.json'

In [4]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, model_max_length=model_max_length)

# Config
config = AutoConfig.from_pretrained(
    model_name, vocab_size=len(tokenizer), pad_token_id=tokenizer.pad_token_id, mask_token_id=tokenizer.mask_token_id, num_labels=nlabels, 
    token_dropout=token_dropout, positional_embedding_type=positional_embedding_type, 
    hidden_size=hidden_size, intermediate_size=intermediate_size, num_attention_heads=num_attention_heads, num_hidden_layers=num_hidden_layers
)

# Model
device = torch.device('cuda:0')
model = EsmForTokenClassification(config)
model = model.to(device)
if model_state_path:
    print("Loading parameters of pretraining model: {}".format(model_state_path))
    if model_state_path.endswith('.bin'):
        model.load_state_dict(torch.load(model_state_path), strict=True)
    elif model_state_path.endswith('.safetensors'):
        load_model(model, filename=model_state_path, strict=False)

Loading parameters of pretraining model: SpliceSitePred/saving_model/mammalian_4096/bs128_lr1e-4_wr0.05_4epochs/checkpoint-20000/model.safetensors


In [5]:
# evaluation metrics
def compute_binary_prauc(true_label, pred_prob):
    """
    Compute PRAUC for single label classification (binary).
    Args:
        true_label(np.array): true labels of sites
        pred_prob(np.array): predicted probabilities of sites, seq len * 1
    """
    precision, recall, _ = precision_recall_curve(true_label, pred_prob)
    prauc = auc(recall, precision)
    return prauc


def compute_ovr_prauc(true_label, pred_prob):
    """
    Compute PRAUC for single label classification (multi-class).
    One vs Rest.
    true_label(np.array): true labels of sites
    pred_prob(np.array): predicted probabilities of sites, seq len * 3
    """
    n_classes = pred_prob.shape[1]
    praucs = []
    for class_idx in range(n_classes):
        prauc = compute_binary_prauc((true_label == class_idx).astype(int), pred_prob[:, class_idx])
        praucs.append(prauc)
    return praucs


def compute_metrics(true_label, pred_prob):
    """
    Compute top-K accuracy for single label (multi-class).
    One vs Rest.
    true_label(np.array): true labels of sites
    pred_prob(np.array): predicted probabilities of sites, seq len * 3
    """
    df = pd.DataFrame(pred_prob)
    df['true_label'] = true_label
    df = df[df["true_label"] != -100]
    counts = df['true_label'].value_counts().to_dict()
    topk_accuracy = [sum((df.sort_values(by=k, ascending=False)[:v])['true_label'] == k) / v for k, v in counts.items()]
    praucs = compute_ovr_prauc(df['true_label'].values, df[[0, 1, 2]].values)
    return topk_accuracy, praucs

In [6]:
with open(test_set_path) as f:
    test_set = json.load(f)

In [7]:
seqs, true_labels = [], []
for ele in tqdm.tqdm(test_set):
    seqs.append(ele['seq'])
    true_labels.extend(ele['label']) 
true_labels = np.array(true_labels)

100%|███████████████████████████████████████████████████████████████████████| 102905/102905 [00:00<00:00, 175101.99it/s]


In [8]:
softmax = nn.Softmax(dim=2)
pred_labels, pred_probs = [], []
model.eval()
with torch.no_grad():
    for seq in tqdm.tqdm(seqs):
        batch = tokenizer(seq, return_tensors='pt', padding=True)
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        model_output = model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            position_ids = None,
            head_mask = None,
            inputs_embeds = None,
            labels = None,
            output_attentions = None,
            output_hidden_states = None,
            return_dict = None
        )
        batch_logits = model_output.logits
        pred_probs.extend(softmax(batch_logits)[0].tolist())
        pred_labels.extend(torch.argmax(batch_logits[0], dim=1).tolist())
pred_probs = np.array(pred_probs)

100%|█████████████████████████████████████████████████████████████████████████| 102905/102905 [1:01:15<00:00, 28.00it/s]


In [9]:
topk_accuracy, praucs = compute_metrics(true_labels, pred_probs)

In [10]:
topk_accuracy_mean = np.mean(topk_accuracy[1:])
prauc_mean = np.mean(praucs[1:])
result_df = pd.DataFrame({'topk_accuracy': topk_accuracy + [topk_accuracy_mean.tolist()], 'prauc': praucs + [prauc_mean.tolist()]})

In [11]:
result_df

,topk_accuracy,prauc
0,0.999974,1.000000
1,0.925957,0.965506
2,0.933917,0.971596
3,0.929937,0.968551
